# NYC Transportation Market Analysis

## 1. Objective

The goal of this notebook is to perform an initial exploratory data analysis (EDA) on the Yellow Taxi and High-Volume For-Hire Vehicle (HVFHV) datasets. We will use pandas to profile a sample of the data.

This analysis will not be exhaustive but will focus on:

1. Schema Profiling: Understanding the columns, data types, and null values.
2. Data cleaning: Identifying "dirty data" (e.g., negative fares, 0-mile trips, outliers).
3. Conformance Strategy: Formulating the business logic required to successfully design the ETL

## 2.Datasets

    Yellow Taxi: yellow_tripdata_2024-{01, 02, 03}.parquet

    FHVHV (Uber/Lyft etc.): fhvhv_tripdata_2024-{01, 02, 03}.parquet

    Zone Lookup: taxi_zone_lookup.csv

## 2. Setup: Imports, Settings, and File Paths

In [1]:
import networkx as nx
import pandas as pd

In [2]:
YELLOW_TAXI_PATH = '../data/raw/taxi/yellow_tripdata_2024-01.parquet'
ZONE_LOOKUP_PATH = '../data/raw/lookups/taxi_zone_lookup.csv'

## 3. Data Loading
Load the three datasets into memory.

In [3]:
yellow_df = pd.read_parquet(YELLOW_TAXI_PATH)
lookup_df = pd.read_csv(ZONE_LOOKUP_PATH)

In [4]:
yellow_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [5]:
yellow_df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               2964624 non-null  int32         
 1   tpep_pickup_datetime   2964624 non-null  datetime64[us]
 2   tpep_dropoff_datetime  2964624 non-null  datetime64[us]
 3   passenger_count        2824462 non-null  float64       
 4   trip_distance          2964624 non-null  float64       
 5   RatecodeID             2824462 non-null  float64       
 6   store_and_fwd_flag     2824462 non-null  object        
 7   PULocationID           2964624 non-null  int32         
 8   DOLocationID           2964624 non-null  int32         
 9   payment_type           2964624 non-null  int64         
 10  fare_amount            2964624 non-null  float64       
 11  extra                  2964624 non-null  float64       
 12  mta_tax                29646

In [6]:
yellow_df['RatecodeID'].isna().sum()

np.int64(140162)

In [7]:
null_counts = yellow_df.isnull().sum()

columns_with_nulls = null_counts[null_counts > 0]

In [8]:
columns_with_nulls

passenger_count         140162
RatecodeID              140162
store_and_fwd_flag      140162
congestion_surcharge    140162
Airport_fee             140162
dtype: int64

In [9]:
yellow_df.fillna({'RatecodeID': 99.0}, inplace=True)

In [10]:
yellow_df.fillna({'passenger_count': 0.0}, inplace=True)

In [11]:
yellow_df['RatecodeID'] = yellow_df['RatecodeID'].astype('int32')
yellow_df['passenger_count'] = yellow_df['passenger_count'].astype('int32')

yellow_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,0,3.18,99,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,0,4.00,99,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,0,3.33,99,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,0,3.06,99,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN


In [12]:
yellow_df = yellow_df[yellow_df['tpep_pickup_datetime'] < yellow_df['tpep_dropoff_datetime']]
yellow_df = yellow_df[yellow_df['tpep_pickup_datetime'].dt.year == 2024 & yellow_df['tpep_dropoff_datetime'].dt.year == 2024]

In [13]:
yellow_df.shape

(2963754, 19)

In [14]:
yellow_df.loc[:,'trip_distance_km'] = round(yellow_df.loc[:,'trip_distance'] * 1.60934, 2)

/tmp/ipykernel_124378/242642743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yellow_df.loc[:,'trip_distance_km'] = round(yellow_df.loc[:,'trip_distance'] * 1.60934, 2)


In [15]:
yellow_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_distance_km
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0,2.77
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0,2.90
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0,7.56
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0,2.25
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,0,3.18,99,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN,5.12
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,0,4.00,99,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN,6.44
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,0,3.33,99,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN,5.36
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,0,3.06,99,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN,4.92


In [18]:
yellow_df['VendorID'].value_counts()

VendorID
2    2234529
1     729017
6        208
Name: count, dtype: int64

In [19]:
yellow_df['passenger_count'].value_counts()

passenger_count
1    2188176
2     405030
0     171446
3      91246
4      51963
5      33501
6      22332
8         51
7          8
9          1
Name: count, dtype: int64

In [44]:
another = yellow_df[yellow_df['passenger_count'].between(0, 6)]


In [45]:
another

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_distance_km
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0,2.77
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0,2.90
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0,7.56
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0,2.25
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,0,3.18,99,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN,5.12
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,0,4.00,99,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN,6.44
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,0,3.33,99,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN,5.36
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,0,3.06,99,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN,4.92


In [46]:
another['passenger_count'].value_counts()

passenger_count
1    2188176
2     405030
0     171446
3      91246
4      51963
5      33501
6      22332
Name: count, dtype: int64

In [47]:
another

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_distance_km
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0,2.77
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0,2.90
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0,7.56
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0,2.25
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,0,3.18,99,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN,5.12
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,0,4.00,99,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN,6.44
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,0,3.33,99,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN,5.36
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,0,3.06,99,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN,4.92


In [48]:
another['store_and_fwd_flag'].value_counts()

store_and_fwd_flag
N    2812584
Y      11059
Name: count, dtype: int64

In [49]:
another['payment_type'].value_counts()

payment_type
1    2318884
2     438560
0     140051
4      46626
3      19573
Name: count, dtype: int64

In [52]:
another['VendorID'].value_counts()

VendorID
2    2234470
1     729016
6        208
Name: count, dtype: int64

In [53]:
another['PULocationID'].value_counts()

PULocationID
132    145176
161    143447
237    142682
236    136433
162    106691
        ...  
105         1
111         1
44          1
204         1
109         1
Name: count, Length: 260, dtype: int64

In [54]:
another_cleaned = another[another['PULocationID'] < 264]

In [68]:
fee_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'congestion_surcharge', 'Airport_fee', 'improvement_surcharge']

In [69]:
another_cleaned[fee_cols].min()

fare_amount             -800.00
extra                     -7.50
mta_tax                   -0.50
tip_amount               -80.00
tolls_amount             -80.00
total_amount            -801.00
congestion_surcharge      -2.50
Airport_fee               -1.75
improvement_surcharge     -1.00
dtype: float64

In [75]:
another_cleaned['']

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_distance_km
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0,2.77
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0,2.90
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0,7.56
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0,2.25
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0,1.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,0,3.18,99,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN,5.12
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,0,4.00,99,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN,6.44
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,0,3.33,99,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN,5.36
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,0,3.06,99,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN,4.92


In [76]:
lookup_df

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN


In [110]:
duplicates = lookup_df.groupby(['Zone', 'service_zone'])['Borough'].count().reset_index().rename(columns = {'Borough': 'count'}).sort_values(by = ['count'], ascending = False)

In [117]:
import networkx as nx

In [113]:
df_edges = another_cleaned[['PULocationID', 'DOLocationID']].dropna()

In [114]:
df_edges = df_edges.astype(int)

In [115]:
edge_list = df_edges.values.tolist()

In [118]:
G = nx.DiGraph()

In [119]:
G.add_edges_from(edge_list)

In [122]:
pagerank_scores = nx.pagerank(G, alpha = 0.85)

In [124]:
df_scores = pd.DataFrame.from_dict(
    pagerank_scores,
    orient = 'index',
    columns = ['pagerank_scores']
)

In [125]:
df_scores.index.name = 'LocationID'

In [127]:
df_results = lookup_df.merge(df_scores, on = 'LocationID')

In [129]:
df_results_sorted = df_results.sort_values(by = 'pagerank_scores', ascending = False)

In [135]:
df_results_sorted[['Zone', 'Borough', 'pagerank_scores']]

,Zone,Borough,pagerank_scores
74,East Harlem South,Manhattan,0.008241
136,Lenox Hill East,Manhattan,0.007864
83,Eltingville/Annadale/Prince's Bay,Staten Island,0.007816
261,Outside of NYC,NaN,0.007176
75,East New York,Brooklyn,0.007117
...,...,...,...
4,Arden Heights,Staten Island,0.000708
102,Governor's Island/Ellis Island/Liberty Island,Manhattan,0.000699
1,Jamaica Bay,Queens,0.000670
98,Freshkills Park,Staten Island,0.000638
